In [3]:
import sys; sys.path.append('..')
from feature_analysis3 import *

In [4]:
seer_model_weights_root = 'https://dl.fbaipublicfiles.com/vissl/model_zoo/'

seer_model_weights = {
    'RegNet-32Gf-SEER': 'seer_regnet32d/seer_regnet32gf_model_iteration244000.torch',
    'RegNet-32Gf-SEER-INFT': 'seer_finetuned/seer_regnet32_finetuned_in1k_model_final_checkpoint_phase78.torch',
    'RegNet-64Gf-SEER': 'seer_regnet64/seer_regnet64gf_model_final_checkpoint_phase0.torch',
    'RegNet-64Gf-SEER-INFT': 'seer_finetuned/seer_regnet64_finetuned_in1k_model_final_checkpoint_phase78.torch',
    'RegNet-128Gf-SEER': ('swav_ig1b_regnet128Gf_cnstant_bs32_node16_sinkhorn10_proto16k_syncBN64_warmup8k/' + 
                          'model_final_checkpoint_phase0.torch'),
    'RegNet-128Gf-SEER-INFT': 'seer_finetuned/seer_regnet128_finetuned_in1k_model_final_checkpoint_phase78.torch',
    'RegNet-256Gf-SEER': ('swav_ig1b_cosine_rg256gf_noBNhead_wd1e5_fairstore_bs16_node64_sinkhorn10_proto16k' +
                          '_apex_syncBN64_warmup8k/' + 'model_final_checkpoint_phase0.torch'),
    'RegNet-256Gf-SEER-INFT': 'seer_finetuned/seer_regnet256_finetuned_in1k_model_final_checkpoint_phase38.torch'
}

seer_model_weights = {key: seer_model_weights_root + value for (key, value) in seer_model_weights.items()}

seer_weight_paths = {key: os.path.join('custom/vissl_weights', key + '.torch')
                                       for (key, value) in seer_model_weights.items()}

seer_config_names = {
    'RegNet-32Gf-SEER': 'regnet32Gf',
    'RegNet-32Gf-SEER-INFT': 'regnet32Gf',
    'RegNet-64Gf-SEER': 'regnet64Gf_3',
    'RegNet-64Gf-SEER-INFT': 'regnet64Gf_3',
    'RegNet-128Gf-SEER': 'regnet128Gf',
    'RegNet-128Gf-SEER-INFT': 'regnet128Gf',
    'RegNet-256Gf-SEER': 'regnet256Gf_1',
    'RegNet-256Gf-SEER-INFT': 'regnet256Gf_1',
}

In [5]:
seer_model_weights.keys()

dict_keys(['RegNet-32Gf-SEER', 'RegNet-32Gf-SEER-INFT', 'RegNet-64Gf-SEER', 'RegNet-64Gf-SEER-INFT', 'RegNet-128Gf-SEER', 'RegNet-128Gf-SEER-INFT', 'RegNet-256Gf-SEER', 'RegNet-256Gf-SEER-INFT'])

In [6]:
#!wget -O custom/vissl_weights/regnet_32Gf_SEER.torch https://dl.fbaipublicfiles.com/vissl/model_zoo/seer_regnet32d/seer_regnet32gf_model_iteration244000.torch

In [7]:
model_name = 'RegNet-32Gf-SEER'

In [8]:
from torchvision.datasets.utils import download_url
download_url(seer_model_weights[model_name], 'custom/vissl_weights', model_name + '.torch')

  0%|          | 0/1430678251 [00:00<?, ?it/s]

In [14]:
?download_url

Signature:
download_url(
    url: str,
    root: str,
    filename: Union[str, NoneType] = None,
    md5: Union[str, NoneType] = None,
    max_redirect_hops: int = 3,
) -> None
Docstring:
Download a file from a url and place it in root.

Args:
    url (str): URL to download file from
    root (str): Directory to place downloaded file in
    filename (str, optional): Name to save the file under. If None, use the basename of the URL
    md5 (str, optional): MD5 checksum of the download. If None, do not check
    max_redirect_hops (int, optional): Maximum number of redirect hops allowed
File:      /storage/colinconwell/miniconda3/envs/synthese/lib/python3.7/site-packages/torchvision/datasets/utils.py
Type:      function


In [9]:
from vissl.utils.hydra_config import compose_hydra_configuration, convert_to_attrdict

model_config = seer_config_names[model_name]

cfg = ['config=benchmark/linear_image_classification/imagenet1k/eval_resnet_8gpu_transfer_in1k_linear', 
       '+config/benchmark/linear_image_classification/imagenet1k/models={}'.format(model_config), 
       'config.MODEL.WEIGHTS_INIT.PARAMS_FILE=custom/vissl_weights/{}.torch'.format(model_name)]

cfg = compose_hydra_configuration(cfg)
_, cfg = convert_to_attrdict(cfg)

In [10]:
from vissl.models import build_model
model = build_model(cfg.MODEL, cfg.OPTIMIZER)
model = model.eval()

In [11]:
from classy_vision.generic.util import load_checkpoint
from vissl.utils.checkpoint import init_model_from_consolidated_weights

# Load the checkpoint weights.
weights = load_checkpoint(checkpoint_path=cfg.MODEL.WEIGHTS_INIT.PARAMS_FILE)

# Initializei the model with the simclr model weights.
init_model_from_consolidated_weights(config=cfg, model=model, state_dict=weights, skip_layers = [],
                                     state_dict_key_name="classy_state_dict")

print("Weights loaded!")

Weights loaded!


In [12]:
print(cfg.MODEL.WEIGHTS_INIT.PARAMS_FILE)

custom/vissl_weights/RegNet-32Gf-SEER.torch


In [9]:
import torchvision.transforms as transforms

image_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [10]:
target_imageset = 'oasis'
model_name = 'RegNet-32GF-SEER'
train_type = 'seer'
model_string = '_'.join([model_name, train_type])
model_option = {'model_name': model_name,
                'train_type': train_type}

In [11]:
image_data = load_image_data(target_imageset)
response_data = load_response_data(target_imageset)

In [12]:
stimulus_loader = get_stimulus_loader(image_data.image_path, image_transforms)

In [ ]:
stimulus_features = get_all_feature_maps(model, inputs = stimulus_loader)
stimulus_features = get_feature_map_srps(stimulus_features, delete_originals = True)

In [ ]:
reg_results = get_regression_results(model_option, stimulus_features, response_data, alpha_values = [1000])

In [ ]:
max_transform(reg_results[reg_results['image_type'] == 'Combo'], group_vars = ['measurement', 'image_type'])

In [ ]:
from plotnine import *
from plotnine import options
options.figure_size = (10,5)

In [ ]:
plot_data = reg_results[(reg_results['image_type'] == 'Combo') & (reg_results['score_type'] == 'pearson_r')]
ggplot(plot_data, aes(x = 'model_layer_index', y = 'score')) + geom_line() + facet_wrap('~measurement')

In [14]:
stimulus_features_x = get_all_feature_maps(model, inputs = stimulus_loader, 
                                          layers_to_retain = ['BatchNorm2d-68'])
stimulus_features_x = get_feature_map_srps(stimulus_features_x, delete_originals = True)

Feature Extraction (Batch):   0%|          | 0/15 [00:00<?, ?it/s]

SRP Extraction (Layer):   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
reg_results_x = get_regression_results(model_option, stimulus_features_x, response_data)

Regression (Layer):   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
max_transform(reg_results_x, group_vars = ['score_type','image_type','measurement'])

,model,train_type,model_layer_index,model_layer,measurement,image_type,score_type,score,alpha
0,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Object,ev_score,0.507092,1000.000000
1,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Object,pearson_r,0.712403,1778.279410
2,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Person,ev_score,0.446604,3162.277660
3,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Person,pearson_r,0.671089,5623.413252
4,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Animal,ev_score,0.317382,5623.413252
5,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Animal,pearson_r,0.565368,10000.000000
6,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Scene,ev_score,0.629283,1000.000000
7,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Scene,pearson_r,0.793887,1778.279410
8,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,ev_score,0.505175,5623.413252
9,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.712339,5623.413252


In [20]:
reg_results_x[(reg_results_x['image_type'] == 'Combo') & (reg_results_x['score_type'] == 'pearson_r')]

,model,train_type,model_layer_index,model_layer,measurement,image_type,score_type,score,alpha
201,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.579972,0.100000
203,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.580006,0.177828
205,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.580067,0.316228
207,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.580176,0.562341
209,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,valence,Combo,pearson_r,0.580368,1.000000
...,...,...,...,...,...,...,...,...,...
741,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,beauty,Combo,pearson_r,0.788889,10000.000000
743,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,beauty,Combo,pearson_r,0.784286,17782.794100
745,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,beauty,Combo,pearson_r,0.774947,31622.776602
747,RegNet-32GF-SEER,seer,1,BatchNorm2d-68,beauty,Combo,pearson_r,0.761687,56234.132519
